In [ ]:
#https://nathancooper.io/i-am-a-nerd/code/summarization/deep-learning/seq2seq/2020/12/26/Improved_Code_Commenter.html
import pandas as pd
import json
import re
from tqdm.auto import tqdm

In [ ]:
#pd.set_option('display.max_colwidth', -1)

In [ ]:
#methods_df = pd.read_json('funcom_test/comments.json')
with open("comments.json") as file:
    dict_comments = json.load(file)
    
with open("functions.json") as file:
    dict_methods = json.load(file)

In [ ]:
df_comments = pd.DataFrame(dict_comments.items(), columns = ['function_id', 'comments'])
df_methods = pd.DataFrame(dict_methods.items(), columns = ['function_id', 'methods'])

In [ ]:
print(df_comments.shape)
print(df_methods.shape)

In [ ]:
import os
list(dict_comments)[500000] 

In [ ]:
print(df_comments.head())
print(df_methods.head())

In [ ]:
dict_df = {'Methods': df_methods['methods'][400000:450000], 'Docstring': df_comments['comments'][400000:450000]}

In [ ]:
methods_df=pd.DataFrame(data=dict_df)

In [ ]:
methods_df.head()

In [ ]:
methods_df.to_csv('original_data.csv', sep=',')

In [ ]:
methods_df.shape

In [ ]:
# From https://stackoverflow.com/a/27084708/5768407
# def is_ascii(s):
#     '''
#     Determines if the given string contains only ascii characters

#     :param s: the string to check
#     :returns: whether or not the given string contains only ascii characters
#     '''
#     try:
#         s.encode(encoding='utf-8').decode('ascii')
#     except UnicodeDecodeError:
#         return False
#     else:
#         return True

In [ ]:
# processed_df = methods_df[methods_df['Methods'].apply(lambda x: is_ascii(x))]
# processed_df = methods_df[methods_df['Docstring'].apply(lambda x: is_ascii(x))]

In [ ]:
processed_df=methods_df

In [ ]:
def remove_comments_inside_mthd(mthd: str) -> str:
    lines = mthd.split("\n")
    eachLine = []
    for line in lines:
        if "//" in line:
            continue
        else:
            eachLine.append(line)
    
    return "\n".join(eachLine)

In [ ]:
processed_df.Methods = processed_df.Methods.apply(remove_comments_inside_mthd)
processed_df.tail()

In [ ]:
def remove_jdocs(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Remove the JavaDocs leaving only the description of the comment

    :param df: the pandas dataframe to remove the JavaDocs from
    :returns: a new pandas dataframe with the JavaDocs removed
    '''
    methods = []
    comments = []
    for i, row in tqdm(list(df.iterrows())):
        comment = row["Docstring"]
        # Remove {} text in comments from https://stackoverflow.com/questions/14596884/remove-text-between-and-in-python/14598135
        comment = re.sub("([\{\[]).*?([\)\}])", '', comment)
        
        
        cleaned = []
        for line in comment.split('\n'):
            if "@" in line: break
            cleaned.append(line)
        comments.append('\n'.join(cleaned))
        methods.append(row["Methods"])
    new_df = pd.DataFrame(zip(methods, comments), columns = ["Methods", "Docstring"])

    return new_df

In [ ]:
processed_df = remove_jdocs(processed_df)
processed_df.tail()

In [ ]:
# collapse
def clean_html(cmt: str) -> str:
    '''
    Remove any HTML tags from a given comment

    :param cmt: the comment to remove any HTML tags from
    :returns: the comment with any HTML tags removed
    '''
    result = re.sub(r"<.?span[^>]*>|<.?code[^>]*>|<.?p[^>]*>|<.?hr[^>]*>|<.?h[1-3][^>]*>|<.?a[^>]*>|<.?b[^>]*>|<.?blockquote[^>]*>|<.?del[^>]*>|<.?dd[^>]*>|<.?dl[^>]*>|<.?dt[^>]*>|<.?em[^>]*>|<.?i[^>]*>|<.?img[^>]*>|<.?kbd[^>]*>|<.?li[^>]*>|<.?ol[^>]*>|<.?pre[^>]*>|<.?s[^>]*>|<.?sup[^>]*>|<.?sub[^>]*>|<.?strong[^>]*>|<.?strike[^>]*>|<.?ul[^>]*>|<.?br[^>]*>", "", cmt)
    return result

In [ ]:
processed_df.Docstring = processed_df.Docstring.apply(clean_html)
processed_df.tail()

In [ ]:
processed_df.shape

In [ ]:
# collapse
processed_df = processed_df.applymap(lambda x: ' '.join(x.split()).lower())

#processed_df = processed_df[~(processed_df['Docstring'] == '')]

#processed_df = processed_df[~processed_df['Docstring'].duplicated()]

In [ ]:
processed_df.shape

In [ ]:
#processed_df = processed_df[~(processed_df['Docstring'] == '')]
#processed_df.tail()
processed_df.shape

In [ ]:
def remove_special_characters(cmt: str) -> str:
    result = re.sub(r"\*|\/|\.", "", cmt)
    return result

In [ ]:
processed_df.Docstring = processed_df.Docstring.apply(remove_special_characters)
processed_df.Docstring.tail()

In [ ]:
processed_df.shape

In [ ]:
processed_df['code_tokens'] = processed_df.Methods.apply(lambda x: x.split())
processed_df['docstring_tokens'] = processed_df.Docstring.apply(lambda x: x.split())

In [ ]:
processed_df.head()

In [ ]:
#values = []
with open('test.jsonl', 'a') as file:
    for i, row in processed_df.iterrows(): 
        val = {"code": row['Methods'], "docstring": row['Docstring'], 
               "code_tokens": row['code_tokens'], "docstring_tokens": row['docstring_tokens']}
        
        json.dump(val, file)
        file.write('\n')
        #values.append(val)